In [1]:
import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
import pandas as pd
from utils.utils import get_paths
from utils.pikeBot_chess_utils import PikeBotModelWrapper, Pikebot
from evaluation_utils import load_pgn, set_engine_history, get_move_evaluation_pairs
from enrich_pgn import add_move_evaluation

In [2]:
config=get_paths()

model = PikeBotModelWrapper(
    config['pikebot'],
    config['preprocessing_params_path']
)
engine = Pikebot(
    model,
    None,
    config["stockfish_path"],
)


In [3]:
used_dir = 'results\MaiaChess_Pikebot_bias-openings_-2_Wed_Nov_13_22-04-25_2024'
game_dir = os.path.join(used_dir, 'games')
if not os.path.isdir(game_dir):
    print("not game directory!")
else:
    os.makedirs(os.path.join(used_dir, 'enriched_games'), exist_ok=True)
    os.makedirs(os.path.join(used_dir, 'move_predictions'), exist_ok=True)

In [5]:
for file_name in os.listdir(game_dir):
    if file_name.endswith('pgn'):
        path = os.path.join(game_dir, file_name)
        games = load_pgn(path)
        all_evaluations = list()
        with open(os.path.join(used_dir, 'enriched_games', file_name), 'w') as output_file:
            for game in games:
                enriched_game, move_predictions = add_move_evaluation(game, engine, model)
            output_file.write(enriched_game)
            output_file.write('\n\n')
            all_evaluations.extend(move_predictions)
            
        all_evaluations_df = pd.DataFrame(all_evaluations)
        all_evaluations_df.to_csv(os.path.join(used_dir, "move_predictions", f"{file_name[:-4]}.csv"), index=False)